In [1]:
### HILBERT LOCK ###
#
# Jon's "tracer bullet" April 14, 2025
#
# jonathan.kaufman@quantumburgh.com
#
# Start with a weak combination lock, and then gradually
# dial down its weakness until its energy landscape vanishes
# everywhere except at a single point: the combination.
# And maybe suppose n-dials-of-k (where k is prime).

In [27]:
from itertools import product
import networkx as nx

class HLock:
        
    def __init__(self):
        #algebraic group Z_3^5
        self.k = 5 # must be prime if assuming field axioms
        self.n = 3
        self.code = (0,0,0) # structure is isomorphic to any code
        self.degeneracy = [1, 6, 18, 32, 36, 24, 8] # triple self convolution of Z_5
        self.skeleton = nx.Graph() # see self.moult()
        self.moult()
    
    def norm(self, setting: tuple) -> int:
        #Banach space
        #Manhattan distance
        #zero = (0,0,0)
        s = setting # abreviation going forward
        a, b, c = s
        d = lambda x: min(x, self.k-x)
        return d(a) + d(b) + d(c)

    def index(self, s: tuple) -> int:
        return s[0] + 5*s[1] + 25*s[2]
    
    def add(self, s1: tuple, s2: tuple) -> tuple:
        #group operation
        #modulo k
        return tuple((x + y) % self.k for x, y in zip(s1, s2))
    
    def neg(self, s: tuple) -> tuple:
        #group inverse
        k = self.k
        a, b, c = s
        return tuple(((k-a)%k, (k-b)%k, (k-c)%k))
    
    def diff(self, s1: tuple, s2: tuple) -> tuple:
        #add a neg
        return self.add(s1,self.neg(s2))
    
    def metric(self, s1: tuple, s2: tuple) -> int:
        #metric space
        return self.norm(self.diff(s1, s2))
    
    def mult(self, a: int, s: tuple) -> tuple:
        #algebraic ring
        #scalar * vector
        k = self.k
        x, y, z = s
        return tuple(((a*x)%k, (a*y)%k, (a*z)%k))

    def dot(self, s1: tuple, s2: tuple) -> int:
        #pseudo-Hilbert
        #inner (dot) product
        k = self.k
        a, b, c = s1
        x, y, z = s2
        return (((a*x)%k + (b*y)%k + (c*z)%k)%k)
    
    def dual(self, s: tuple) -> tuple:
        #bra (as in bra-ket)
        return s
    
    def weak(self, s: tuple) -> int:
        #error (potential energy)
        return self.metric(self.code, s)

    def stratum(self, level: int):
        #energy level
        k = self.k
        s = set()
        for xyz in product(range(k), repeat=3):
            if self.weak(xyz) == level:
                s.add(xyz)
        return s

    def neighbors(self, s: tuple) -> set:
        #Manhattan distance = 1
        k = self.k
        a, b, c = s
        return set((((a+1)%k,b,c),((a-1)%k,b,c),(a,(b+1)%k,c),(a,(b-1)%k,c),(a,b,(c+1)%k),(a,b,(c-1)%k)))

    def moult(self):
        #refresh skeleton
        G  = self.skeleton
        kk = self.k + int(self.k/2)
        def f(v):
            G.add_node(v)
            [G.add_edge(v,n) for n in self.neighbors(v)]
        [[f(s) for s in self.stratum(rung)] for rung in range(kk)]
        return G
    
    def solution(self, s: tuple) -> list:
        return list(nx.shortest_path(self.skeleton, source=s, target=self.code))
    
    def cut(self, s1: tuple, s2: tuple):
        return self.skeleton.remove_edge(s1,s2)
    
    @classmethod
    def test(cls):
        #method tests
        l = cls() #instance
        p = '\033[92m'+'pass'+'\033[0m'
        f = '\033[91m'+'fail'+'\033[0m'
        print('norm():        ', p if l.norm((1,2,3))            ==       5  else f)
        print('add():         ', p if l.add((1,2,3), (2,3,1))    == (3,0,4)  else f)
        print('neg():         ', p if l.neg((1,2,3))             == (4,3,2)  else f)
        print('mult():        ', p if l.mult(2, (1,2,3))         == (2,4,1)  else f)
        print('diff():        ', p if l.diff((1,2,3), (2,1,3))   == (4,1,0)  else f)
        print('metric():      ', p if l.metric((1,2,3), (2,2,3)) ==       1  else f)
        print('dot():         ', p if l.dot((0,1,1), (1,2,3))    ==       0  else f)
        print('dual():        ', p if l.dual((1,2,3))            == (1,2,3)  else f)
        print('weak():        ', p if l.weak((2,2,1))            ==       5  else f)
        print('stratum():     ', p if len(l.stratum(4))          ==      36  else f)
        print('neighbors():   ', p if (0, 0, 3) in l.neighbors((0,0,4))      else f)
        print('moult():       ', p if len(l.moult().nodes)       ==     125  else f)
        print('moult():       ', p if len(l.moult().edges)       ==     375  else f)
        print('solution():    ', p if len(l.solution((3,4,2)))   ==       6  else f)
        l.cut((0,0,0),(0,0,1))
        print('cut():         ', p if len(l.solution((0,0,1)))   ==       4  else f)
        q = [l.weak(j) for j in l.solution((0,0,1))]
        print('test017:       ', p if [1,2,1,0]                  ==       q  else f)
        q = [l.weak(j) for j in l.solution((2,2,2))]
        print('test018:       ', p if [6,5,4,3,2,1,0]            ==       q  else f)

In [28]:
l= HLock()
l.test()

norm():         pass
add():          pass
neg():          pass
mult():         pass
diff():         pass
metric():       pass
dot():          pass
dual():         pass
weak():         pass
stratum():      pass
neighbors():    pass
moult():        pass
moult():        pass
solution():     pass
cut():          pass
test017:        pass
test018:        pass
